# Playground

In [1]:
import torch
from pathlib import Path

from uda import CC359, CC359Config

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

data_dir = Path("/tmp/data/CC359")

Using device: cuda:0


In [2]:
dataset_config = CC359Config(vendor="GE_3", fold=0, patch_size=(96, 128, 128), random_state=42)

dataset = CC359(data_dir, dataset_config)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)  # batch_size 32 <=> 4 subjects

Loading files:   0%|          | 0/20 [00:00<?, ?it/s]

In [3]:
x, y_true = next(iter(data_loader))
print(x.shape)
print(y_true.shape)
print(x.dtype)
print(y_true.dtype)

torch.Size([32, 1, 96, 128, 128])
torch.Size([32, 1, 96, 128, 128])
torch.float32
torch.float32
